In [ ]:
import numpy as np
import plotly.graph_objects as go
import plotly.io as pio
pio.renderers.default='notebook'
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)
import plotly.figure_factory as ff
import sys

In [ ]:
f_a = 200.0  # Given accelerometer frequency - 200Hz
f_m = 5.0    # Given measurement frequency - 5Hz
T = 30;      # Time period for simulation [s]
dt = 0.02;   # Time interval [s]
omega = 0.1; # Given angular velocity - 0.1rad/s

# Create time vector for discrete time model
# To get the size of vector, divide the time period by time interval
# vector size
size = T/dt+1                   
# acceleration time vector with dt = 0.02 sec    
tVec = np.linspace(0.0, 30.0, size)
# Given angular velocity - 0.1rad/s 
omega = 0.1                         
# true acceleration model
a_t = [10*np.sin(omega*tVec[i]) for i in range(tVec.shape[0])]

In [ ]:
# state transition matrix
Phi = np.array([
    [1, dt, -dt**2/2],
    [0, 1, -dt],
    [0, 0, 1]
])

# Gamma (B) matrix
B = np.array([
    [-dt**2/2],
    [-dt],
    [0]
])

# Accelerometer process noise
w_var = 0.0004 # variance [(m/s^2)^2]
w_mean = 0     # mean
W = w_var      # variance matrix

# Obervation model matrix
# H matrix used in z = Hx + v
H = np.array([
    [1, 0],
    [0, 1]
])

# measurement noise mean and variance
eta_mean = np.zeros((2,1)) # mean 
eta_var = [1 0;0 (0.04)^2] # variance. 1m^2 and (4cm/s)^2
V = dot(eta_var,eta_var)   # Variance matrix

# Accelerometer bias
b_mean = 0   # mean
b_var = 0.01 # variance [(m/s^2)^2]

# Initial position
x0_mean = 0  # mean [m]
x0_var = 100 # variance [m^2]

# Initial velocity
v0_mean = 100 # mean [m/s]
v0_var = 1    # variance [(m/s)^2]


# initial covariance matrix
P0 = np.array([
    [x0_var, 0, 0],
    [0, v0_var, 0],
    [0, 0, b_var]
])

In [ ]:
# Propagation
# M_k+1 = phi_k*P_k*phi_k^T+gamma_k*W_K*gamma_k^T
Mk = np.dot(Phi, np.dot(P0, np.transpose(Phi))) + np.dot(B, np.dot(W, np.transpose(B)))
# Update
# P_k = ((M_k)^(-1)+(H_k)^T*(V_k)^(-1)*H_k)^(-1)
Pk = np.linalg.inv(np.linalg.inv(Mk) + np.dot(np.transpose(H), np.dot(np.linalg.inv(V), H)))
# Kalman gain
# K_k = P_k*H_k^T*V_k^(-1)
Kk = np.dot(Pk, np.dot(np.transpose(H), np.linalg.inv(V)))

Ks = [Kk]
Ms = [Mk]
Ps = [Pk]
count = 1

while count < 2 or not np.allclose(Ks[-2], Ks[-1], rtol=1.0e-09, atol=1e-09):
    # M_k+1 = phi_k*P_k*phi_k^T+gamma_k*W_K*gamma_k^T
    Mk = np.dot(Phi, np.dot(Pk, np.transpose(Phi))) + np.dot(B, np.dot(W, np.transpose(B)))
    # P_k = ((M_k)^(-1)+(H_k)^T*(V_k)^(-1)*H_k)^(-1)
    Pk = np.linalg.inv(np.linalg.inv(Mk) + np.dot(np.transpose(H), np.dot(np.linalg.inv(V), H)))
    # K_k = P_k*H_k^T*V_k^(-1)
    Kk = np.dot(Pk, np.dot(np.transpose(H), np.linalg.inv(V)))
    count+=1

    # update vector
    Ks.append(Kk.copy())
    Ms.append(Mk.copy())
    Ps.append(Pk.copy())

In [ ]:
# Functions for MontelCarlo Simulation
def monteCarlo():
    # Phi, Gamma, W, V, P0, H are already defined
    # Initialize true position, velocity, and bias
    p0_t = np.random.normal(x0_mean, x0_var)
    v0_t = np.random.normal(v0_mean, v0_var)
    b_t = np.random.normal(b_mean, b_var)
    
    # initialize the belief values for the position and velocity
    p_c = x0_mean
    v_c = v0_mean
    
    # temp variables to hold the true position and velocity
    p_tt = p0_t
    v_tt = v0_t
    
    # initial values
    x_hat = np.array([
    [0],
    [0],
    [0]]) # x hat
    Pk = np.copy(P0) # Mk
    
    # posteriori
    eta = np.random.normal(eta_mean, eta_var) # GPS noise
    p_gps = p_tt + eta[0,0]                   # position
    v_gps = v_tt + eta[1,1]                   # velocity
    
    
    # Kalman gain
    invterm = np.linalg.inv(np.dot(H, np.dot(P0, np.transpose(H)))+ V)
    K = np.dot(P0, np.dot(np.transpose(H), invterm))
    
    # update intial belief 
    z = np.array([
        [p_gps-p_c],
        [v_gps-v_c]
    ]) # measurement
    R = z - np.dot(H, x_hat)     # residual
    x_hat = x_hat + np.dot(K, R) # x hat
    
    # update Pk
    ikh = np.eye(3) - np.dot(K, H)
    Pk = np.dot(ikh, np.dot(P0, np.transpose(ikh))) +\
     np.dot(K, np.dot(V, np.transpose(K)))
    
    # create vector for plotting
    d_l = [x_hat] # x hat
    d_tl = [np.array([[p0_t - p_c], [v0_t - v_c], [b_t]])] # error
    p_tl = [p0_t] # true position
    v_tl = [v0_t] # true velocity
    p_cl = [p_c]  # computed position
    v_cl = [v_c]  # computed velocity
    P_kl = [np.copy(Pk)] # Pk
    K_l = [np.copy(K)]   # Kalman gain
    R_l = [R]            # residual

    # run the MC through the time vector
    for i  in range(1, tVec.shape[0]):
        if i % 40 == 0: #t_i = t_(i-1)+40*dt
            # get position, velocity, x_hat, Pk, K, residual
            p_c, v_c, p_tt, v_tt, x_hat, Pk, K, res = gpsTimestep(i, b_t, p_c, v_c, p_tt, v_tt, x_hat, Pk, K)
            # update vectors
            d_l.append(x_hat.copy())
            d_tl.append(np.array([[p_tt - p_c], [v_tt - v_c], [b_t]]))
            p_tl.append(p_tt)
            v_tl.append(v_tt)
            p_cl.append(p_c)
            v_cl.append(v_c)
            P_kl.append(np.copy(Pk))
            K_l.append(np.copy(K))
            R_l.append(np.copy(res))
        else:
            p_c, v_c, p_tt, v_tt = accTimestep(i, b_t, p_c, v_c, p_tt, v_tt)
    
    return p_cl, v_cl, p_tl, v_tl, b_t, d_l, P_kl, K_l, R_l, d_tl

# Function to update kalman filter given both GPS and accelerometer
# input:
#   current and previous position and velocity for 
#   accelerometer and truth mode
#   b - bias
#   j - index
#   Pk_prev - Covariance
#   x_hat - posteriori value
# output:
#   position and velocity for both model
#   Kalman gain, residual, covariance, posteriori value
def gpTimestep(j, b, pc_p, vc_p, pt_p, vt_p, x_hat, Pk_p, K):
    # update
    x_bar = np.dot(Phi, x_hat) # priori
    Mk = np.dot(Phi, np.dot(Pk_prev, np.transpose(Phi)))+\
     np.dot(B, np.dot(W, np.transpose(B))) # covariance
    
    # kalman gainn
    invterm = np.linalg.inv(np.dot(H, np.dot(Mk, np.transpose(H)))+ V)
    K = np.dot(Mk, np.dot(np.transpose(H), invterm))
    
    # get position and velocity
    # i: Represents the time vector to index into the true acceleration vector
    pc_i, vc_i, ptrue_i, vtrue_i = accTimestep(j, b, pc_p, vc_p, pt_p, vt_p)
    
    # GPS measurement data
    eta = np.random.normal(eta_mean, eta_var) # noise
    pgps = pt_i + eta[0,0]                    # position
    vgps = vt_i + eta[1,1]                    # velocity

    # measurement model
    zk = np.array([
        [pgps - pc_i],
        [vgps - vc_i]
    ])
    
    # calculate the residual
    R = zk - np.dot(H, x_bar)
    
    # update x_hat
    x_hat = x_bar + np.dot(K, R)
    
    # update covariance
    hvh = np.dot(np.transpose(H), np.dot(np.linalg.inv(V), H)) #H^T*V*H
    Pk = np.linalg.inv(np.linalg.inv(Mk_next) + hvh)
    
    return pc_i, vc_i, pt_i, vt_i, x_hat, Pk, K, R

# Function to compute the position and velocity at ith time stamp by 
# sampling the accelerometer
# input:
#   current and previous position and velocity for 
#   accelerometer and truth mode
#   b - bias
#   j - index
# output: position and velocity for both model
def accTimestep(j, b, pc_p, vc_p, pt_p, vt_p):
    # noise for accelerometer model
    w = np.random.normal(w_mean, w_var)
    
    # accelerometer model
    # a_c(t_j) = a(t_j)+b+w(t_j)
    ac_p = a_t[j-1] + b + w
    
    # i: Represents the time vector to index into the true acceleration vector
    # velocity and position using Euler formula
    vc_i = vc_p + ac_p * dt
    pc_i = pc_p + vc_p * dt + ac_p * dt**2/2
    
    # true velocity and position using Euler formula
    vt_i = vt_p + a_t[j-1] * dt
    pt_i = pt_p + vt_p * dt + a_t[j-1] * dt**2/2
    
    return pc_i, vc_i, pt_i, vt_i

In [ ]:
p_cl, v_cl, p_tl, v_tl, b_t, d_l, P_kl, K_l, R_l, d_tl = monteCarlo()

# position vectors
p_est = [d_l[i][0,0] + p_cl[i] for i in range(len(d_l))] # position estimate
p_err = [p_tl[i] - p_est[i] for i in range(len(p_cl))]   # position error
# standard deviation of position
p_std = [np.sqrt(P_kl[i][0,0]) for i in range(len(P_kl))]
# position with sigma plus
p_sp = [p_est[i] + p_std[i] for i in range(len(p_est))]
# position with sigma minus
p_sm = [p_est[i] - p_std[i] for i in range(len(p_est))]

# velocity vectors
v_est = [d_l[i][1,0] + v_cl[i] for i in range(len(d_l))] # velocity estimate
v_err = [v_tl[i] - v_est[i] for i in range(len(v_cl))]   # velocity error
# standard deviation of velocity
v_std = [np.sqrt(P_kl[i][1,1]) for i in range(len(P_kl))]
# velocity with sigma plus
v_sp = [v_est[i] + v_std[i] for i in range(len(v_est))]
# velocity with sigma minus
v_sm = [v_est[i] - v_std[i] for i in range(len(v_est))]

# bias vectors
b_est = [d_l[i][2,0] for i in range(len(d_l))]    # bias estimate
b_err = [b_est[i] - b_t for i in range(len(d_l))] # bias error
# bias standard deviation
b_std = [np.sqrt(P_kl[i][2,2]) for i in range(len(P_kl))]
# bias with sigma plus
b_sp = [b_est[i] + b_std[i] for i in range(len(b_est))]
# bias with sigma minus
b_sm = [b_est[i] - b_std[i] for i in range(len(b_est))]

# Bias plot - Estimated VS. Actual
fig1 = go.Figure()
fig1.add_trace(go.Scatter(x=tVec, y=b_est, name="Estimated", line=dict(color='indigo')))
fig1.add_trace(go.Scatter(x=tVec, y=b_sp, name="$+ \sigma$", line=dict(color='springgreen', width=1, dash='dash')))
fig1.add_trace(go.Scatter(x=tVec, y=b_sm, name="$-\sigma$", line=dict(color='deepskyblue', width=1, dash='dash')))
fig1.add_trace(go.Scatter(x=tVec, y=[b_t for i in range(len(dxlist))], name="Actual", line=dict(color='crimson'))
fig1.update_layout(title={'text':r"$\text{Bias - Estimated VS. Actual}$",'xanchor':'center','yanchor':'top','y':0.9,'x':0.48}, xaxis_title=r"$\text{Time (} s \text{)}$", yaxis_title=r"$\text{Bias (}m/s^2 \text{)}$)")
fig1.show(renderer="colab")

# Bias error plot
fig2 = go.Figure()
fig2.add_trace(go.Scatter(x=tVec, y=b_err, name="Error"), line=dict('crimson'))
fig2.update_layout(title={'text':r"$\text{Error - Bias}$",'xanchor':'center','yanchor':'top','y':0.9,'x':0.48}, xaxis_title=r"$\text{Time (} s \text{)}$", yaxis_title=r"$\text{Bias Error (} m/s^2 \text)$")
fig2.show(renderer="colab")

# Position plot - Estimated VS. Actual
fig3 = go.Figure()
fig3.add_trace(go.Scatter(x=tVec, y=p_tl, name="Actual", line=dict(color='indigo',width=4)))
fig3.add_trace(go.Scatter(x=tVec, y=p_est, name="Estimated", linen=dict(color='crimson')))
fig3.add_trace(go.Scatter(x=tVec, y=p_sp, name="$+ \sigma$", line=dict(color='springgreen', width=1, dash='dash')))
fig3.add_trace(go.Scatter(x=tVec, y=p_sm, name="$-\sigma$", line=dict(color='deepskyblue', width=1, dash='dash')))
fig3.update_layout(title={'text':r"$\text{Position - Estimated VS. Actual}$",'xanchor':'center','yanchor':'top','y':0.9,'x':0.48}, xaxis_title=r"$\text{Time (} s \text{)}$", yaxis_title=r"$\text{Position (} m \text{)}$")
fig3.show(renderer="colab")

# Position error plot
fig4 = go.Figure()
fig4.add_trace(go.Scatter(x=tVec, y=p_err, name="Error"))
fig4.update_layout(title={'text':r"$\text{Error - Position}$",'xanchor':'center','yanchor':'top','y':0.9,'x':0.48}, xaxis_title=r"$\text{Time (} s \text{)}$", yaxis_title=r"$\text{Position Error (} m \text{)}$")
fig4.show(renderer="colab")

# Velocity plot - Estimated VS. Actual
fi5 = go.Figure()
fig5.add_trace(go.Scatter(x=tVec, y=v_l, name="Estimated", line=dict(color='indigo', width=4)))
fig5.add_trace(go.Scatter(x=tVec, y=v_tl, name="Actual", line=dict(color='crimson')))
fig5.add_trace(go.Scatter(x=tVec, y=v_sp, name="$+ \sigma$", line=dict(color='springgreen', width=1, dash='dash')))
fig5.add_trace(go.Scatter(x=tVec, y=v_sm, name="$-\sigma$", line=dict(color='deepskyblue', width=1, dash='dash')))
fig5.update_layout(title={'text':r"$\text{Velocity - Estimated VS. Actual}$",'xanchor':'center','yanchor':'top','y':0.9,'x':0.48}, xaxis_title=r"$\text{Time (} s \text{)}$", yaxis_title=r"$\text{Velocity (} m/s \text{)}$")
fig5.show(renderer="colab")

# Velocity error plot
fig6 = go.Figure()
fig6.add_trace(go.Scatter(x=tVec, y=v_err, name="Error"))
fig6.update_layout(title={'text':r"$\text{Error - Velocity}$",'xanchor':'center','yanchor':'top','y':0.9,'x':0.48}, xaxis_title=r"$\text{Time (} s \text{)}$", yaxis_title=r"$\text{Velocity Error (} m/s \text{)}$")
fig6.show(renderer="colab")

In [ ]:
# Monte Carlo simulation function
def MonteCarloSim(l):
    # initialize error vector
    e = np.zeros((l, size, 3))
    # initialize average error vector
    e_avg = np.zeros((size, 3))
    # initialize error variance
    errP = np.zeros((size, 3, 3))
    # initialize orthogonaltiy check matrix
    O = np.zeros((size, 3, 3))
    # initialize residual matrix
    R_m = np.zeros((l, size, 2))
    # initialize residual correlation matrix
    R_cm = np.zeros((2, 2))
    # initialize residual indicies
    m = 100
    q = 101
    # initialize states matrix
    S = np.zeros((l, size, 3))
    # initialize the matrix for predicted pos, vel, bias
    xhat = np.zeros((l, size, 3))
    
    # run Monte Carlo simulation to retrieve errors, states, and residuals
    for i in range(l):
        p_cl, v_cl, p_tl, v_tl, b_t, d_l, Pk_l, K_lt, R_l, d_tl  = monteCarlo()
        # get error matrix
        e[i,:,:] = (np.array(d_tl) - np.array(d_l)).squeeze()
        # get states matrix
        S[i,:,:] = np.array(d_l).squeeze()
        # get residual matrix
        R_m[i,:,:] = np.array(R_l).squeeze()

        xhat[i,:,0] = np.array([d_l[j][0,0] + p_cl[j] for j in range(size)])
        xhat[i,:,1] = np.array([d_l[j][1,0] + v_cl[j] for j in range(size)])
        xhat[i,:,2] = np.array([d_l[j][2,0] for j in range(size)])
    
    # calculate average error
    e_avg = np.sum(e, axis=0) * 1.0/l
    
    # calculate average covariance and orthogonality check matrices
    for t in range(size):
        for i in range(l):
            # covariance
            # calculate difference
            diff = np.reshape(e[i,t,:]-e_avg[t], (3,1))
            # update matrix
            errP[t,:,:] += np.dot(diff,np.transpose(diff))
            
            # orthogonality matrix
            O[t,:,:] += np.dot(diff, np.reshape(xhat[i,t,:], (1,3)))
            
    # scale the covariance data
    errP[:,:,:] *= 1.0/(l-1)
    O *= 1.0/l
    
    # calculate ensemble average correlation of the residuals    
    for i in range(l):
        R_cm += np.dot(np.reshape(R_m[i,m,:], (2,1)),np.reshape(R_m[i,q,:], (1,2)))
    R_cm *= 1.0/l
    
    return e_avg, errP, O, xhat, R_cm

In [ ]:
# Run monte carlo simulation
e_avg, errP, O, xhat, R_cm = MonteCarloSim(10000)

In [ ]:
e_avgx = [eave[i, 0] for i in range(size)] # position average error
e_avgv = [eave[i, 1] for i in range(size)] # velocity average error
e_avgb = [eave[i, 2] for i in range(size)] # bias average error

# Position error graph
fig7 = go.Figure()
fig7.add_trace(go.Scatter(x=size, y=e_avgx, name="Position Error", line=dict(color='crimson', width=4)))
fig7.update_layout(title={'text':r"$\text{Average Position Error}$",'xanchor':'center','yanchor':'top','y':0.9,'x':0.48}, xaxis_title=r"$\text{Time (}s \text{)}$", yaxis_title=r"$\text{Position Error}$")
fig7.show(renderer="colab")

# Velocity error graph
fig8 = go.Figure()
fig8.add_trace(go.Scatter(x=tVec, y=e_avgv, name="Velocity Error", line=dict(color='crimson', width=4)))
fig8.update_layout(title={'text':r"$\text{Average Velocity Error}$",'xanchor':'center','yanchor':'top','y':0.9,'x':0.48}, xaxis_title=r"$\text{Time (}s \text{)}$", yaxis_title=r"$\text{Velocity Error}$")
fig8.show(renderer="colab")

In [ ]:
# covariance difference norm
pdiff = [np.linalg.norm(p_avg[i,:,:] - Ps[i]) for i in range(size)]
fig9 = go.Figure()
fig9.add_trace(go.Scatter(x=tVec, y=pdiff, line=dict(color='crimson'), name="Covariance Difference Norm"))
fig9.update_layout(title={'text':r"$\text{Norm of the Difference Between } P_{avg}(t_i) \text{ and } P(t_i)$",'xanchor':'center','yanchor':'top','y':0.9,'x':0.48}, xaxis_title=r"$\text{Time (}s \text{)}$", yaxis_title=r"$\text{Norm}$")
fig9.show(renderer="colab")

In [ ]:
# covariance difference
p_avga = [p_avg[i, 0, 0] for i in range(size)]
pa = [Ps[i][0,0] for i in range(size)]
fig10 = go.Figure()
fig10.add_trace(go.Scatter(x=tVec, y=p_avga, name="Average",line=dict(color='springgreen')))
fig10.add_trace(go.Scatter(x=tVec, y=pa, name="Covariance",line=dict(color='deepskyblue')))
fig10.update_layout(title={'text':"Difference Between Pavg(t_i)[0,0] and P(t_i)[0,0]",'xanchor':'center','yanchor':'top','y':0.9,'x':0.48}, xaxis_title="$\text{Time (}s \text{)}$", yaxis_title="Difference")
fig10.show(renderer="colab")

In [ ]:
# Orthogonality difference norm
O_n = [np.linalg.norm(O[i,:,:]) for i in range(size)]
fig_o = go.Figure()
fig_o.add_trace(go.Scatter(x=tVec, y=O_n, name="Norm"))
fig_o.update_layout(title={'text':r"$\text{Norm of the Orthogonality Matrix}$",'xanchor':'center','yanchor':'top','y':0.9,'x':0.48}, xaxis_title=r"$\text{Time (}s \text{)}$", yaxis_title=r"$\text{Norm}$")
fig_o.show(renderer="colab")